In [ ]:
import pandas as pd

In [ ]:
train = pd.read_csv('../input/titanicdataset/train.csv')
train.iloc[-5:]

In [ ]:
train.iloc[:5]

In [ ]:
test = pd.read_csv('../input/titanicdataset/test.csv')
test.head(5)

In [ ]:
train.describe()   #numerical features

In [ ]:
train.describe(include=['O'])  #categorial fearures

In [ ]:
import seaborn as sns
sns.lmplot(data=train,x='PassengerId',y='Fare',hue='Survived',)

In [ ]:
sns.scatterplot(data=train,x='PassengerId',y='Pclass',hue='Survived',)

In [ ]:
train[['Pclass','Survived']].groupby('Pclass',as_index=False).mean()

In [ ]:
train[['Sex','Survived']].groupby('Sex',as_index=False).mean()

In [ ]:
train[['Embarked','Survived']].groupby('Embarked',as_index=False).mean()

In [ ]:
sns.scatterplot(data=train,x='PassengerId',y='Parch',hue='Survived',)

In [ ]:
train[['Parch','Survived']].groupby('Parch',as_index=False).mean()

In [ ]:
sns.scatterplot(data=train,x='PassengerId',y='SibSp',hue='Survived',)

In [ ]:
sns.scatterplot(data=train,x='PassengerId',y='Age')

In [ ]:
train[['Survived','SibSp']].groupby('SibSp',as_index=False).mean()

In [ ]:
sns.scatterplot(data=train,x='PassengerId',y='Age',hue='Survived',)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.heatmap(train.isnull())

In [ ]:
sns.heatmap(test.isnull())

In [ ]:
total = train.isnull().sum().sort_values(ascending=False)
percent = (train.isnull().sum()/train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

In [ ]:
total = test.isnull().sum().sort_values(ascending=False)
percent = (test.isnull().sum()/test.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

In [ ]:
import numpy as np
import re
full_data = [train, test]
train["Words_count"] = train['Name'].apply(lambda x: len(x.split()))
test['Words_Count'] = test['Name'].apply(lambda x: len(x.split()))

# Feature that tells whether a passenger had a cabin on the Titanic
train['Has_Cabin'] = train["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
test['Has_Cabin'] = test["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
# Create new feature IsAlone from FamilySize
for dataset in full_data:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
    
# Remove all NULLS in the Embarked column
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')  
    
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())    
train['CategoricalFare'] = pd.qcut(train['Fare'], 4)

for dataset in full_data:
    dataset['Age'] = dataset['Age'].fillna(train['Age'].mean())

def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""
for dataset in full_data:
    dataset['Title'] = dataset['Name'].apply(get_title)
    
    # Group all non-common titles into one single grouping "Rare"
for dataset in full_data:
     dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess', 'Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
     dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
     dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
     dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs') 
 
for dataset in full_data:
    # Mapping Sex
    dataset['Sex'] = dataset['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
    
 # Mapping titles
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
dataset['Title'] = dataset['Title'].map(title_mapping)
dataset['Title'] = dataset['Title'].fillna(0)  

# Mapping Embarked
dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

 # Mapping Fare
dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
dataset['Fare'] = dataset['Fare'].astype(int)
    
   

    


In [ ]:
drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp']
train = train.drop(drop_elements, axis = 1)


In [ ]:
train = train.drop([ 'CategoricalFare'], axis = 1)
test  = test.drop(drop_elements, axis = 1)

In [ ]:
train.head(3)

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# find out the numerical columns in the data
y=train.Survived
X=train.drop('Survived',axis=1)

numerical_column=[col for col in X.columns if X[col].dtype in ['int64','float64']]

#find the categorical column in the data
categorical_column=list(set(X.columns)-set(numerical_column))
print("Numerical columns",numerical_column,'\n','Categorical columns',categorical_column)

In [ ]:
y_train = train['Survived'].ravel()
train = train.drop(['Survived'], axis=1)
x_train = train.values # Creates an array of the train data
x_test = test.values 

In [ ]:
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC

from sklearn.pipeline import Pipeline


my_pipeline = Pipeline(steps=[
                              ('model', RandomForestClassifier(n_estimators=50,
                                                              random_state=0))
                             ])
from sklearn.model_selection import cross_val_score

# Multiply by -1 since sklearn calculates *negative* MAE
scores = -1 * cross_val_score(my_pipeline, x_train, y_train,
                              cv=5,
                              scoring='accuracy')

print("MAE score:\n", scores.mean())

In [ ]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import SVC, LinearSVC
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.naive_bayes import GaussianNB
# from sklearn.linear_model import Perceptron
# from sklearn.linear_model import SGDClassifier
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier
# # use to predict based upon the classes

# from sklearn.model_selection import train_test_split
# # use to split the data to validate

# from sklearn.model_selection import cross_val_score
# # check the validation score

In [ ]:
# # Function to get the cross validation accuracy
# def get_cross_val_score(n,X,y):
#     #n is number of estimators that a model should take 
#     # X is the features 
#     # y is the set target
    
#     # numerical preprocessors with strategy mean
#     numerical_preprocess=SimpleImputer(strategy='mean')
    
#     # categorical preprocessor { we can choose to use label encode on SEX but it cant be considered that what is more significiant
#     # male or female , same apply for Embarked}
#     # there is no data leakage {Target leakage and train test combination leakage}
#     categorical_preprocess=Pipeline(steps=[('imputer',SimpleImputer(strategy='most_frequent')),
#                                       ('HotEncoder',OneHotEncoder())])
    
#     # not use columntransformer to transform column and create parent preprocessor
#     preprocessor=ColumnTransformer([('numerical',numerical_preprocess,numerical_column),
#                             ('categorical',categorical_preprocess,categorical_column)])
    
#     #finally create a pipeline that do the automated task for you
#     global final_pipeline
#     final_pipeline=Pipeline(steps=[('preprocessor',preprocessor),('model',RandomForestClassifier(n_estimators=400, random_state=0))])
    
#     #return the cross validation score 
#     # we only take 5 fold of the traninng data
#     # and doing it in this way help us to make a better model in terms of accuracy 
#     # increase and decrease the cross validation folds to maximize accuracy
#     return cross_val_score(final_pipeline,X,y,cv=5,scoring='accuracy').mean()


In [ ]:
# # for checking what is the best number of n_estimators that can predict the best
# for i in range(400,500,10):
#   print(get_cross_val_score(i,train_X,train_y,valid_X,valid_y))


In [ ]:
# print("The cross validation score for 400 estimators is ",get_cross_val_score(400,X,y))

In [ ]:
# #train the model on training set
# final_pipeline.fit(X,y)
# # make predictions using the pipeline
# # on checking we found out that the model have 87% accuracy on test set 
# test_preds=final_pipeline.predict(test)
# print(final_pipeline.score(X,y))

# #setting the data to submit the file
# output = pd.DataFrame({'PassengerId': test.PassengerId,
#                        'Survived': test_preds})
# output

In [ ]:
# output.to_csv('submission.csv', index=False)